In [1]:
!pip install seqeval

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import pandas as pd
import numpy as np
import os
import json
from seqeval.metrics import f1_score
from sagemaker.pytorch import PyTorchPredictor
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

### Load test set

In [6]:
train_input = pd.read_csv(os.path.join('../data/processed_roberta/', 'test_roberta.csv'), header=None, names=None)
    
val_tags = train_input[train_input.columns[0:45]].values
val_inputs = train_input[train_input.columns[45:90]].values
val_masks = train_input[train_input.columns[90:135]].values
val_segs = train_input[train_input.columns[135:]].values

val_inputs = torch.tensor(val_inputs)
val_tags = torch.tensor(val_tags)
val_masks = torch.tensor(val_masks)
val_segs = torch.tensor(val_segs)

In [10]:
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=32)

In [12]:
batch_num = 32

### Create predictor from endpoint

In [7]:
predictor = PyTorchPredictor('sagemaker-pytorch-2020-05-31-12-23-54-436')

## Evaluate

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# model.eval();

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = predictor(input_ids, token_type_ids=None,
                        attention_mask=input_mask,)
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

***** Running evaluation *****
  Num examples =14388
  Batch size = 32


TypeError: 'PyTorchPredictor' object is not callable